# Scatter Plots, Data Fitting, and Histograms

We are going to load in a different dataset, located in data_linfit.txt. This is a more basic file that is just separated by spaces with no headers but we can read it in just fine with `genfromtxt`

We will make a scatter plot using this data with matplotlib.

In [ ]:
import numpy as np
#note that genfromtxt autmoatically knows that spaces separate the data (that's the default)
x, y = np.genfromtxt('data_linfit.txt',unpack=True)

In [ ]:
import matplotlib.pyplot as plt
#scatter takes x, y arguments. You can also set the marker.
#Colors can be set too, just like in plot()
plt.scatter(x,y, marker='x', color='k')
plt.ylabel('y',fontsize='large')
plt.xlabel('x',fontsize='large')


## Linear fits with Scipy

As we often want to do in science, we will fit this data to a line using linear regression.

The toolkit Scipy has a lot of useful statistics tools. This linear regression function, located in the stats package, outputs an object that inclues all the fit values and their associated statistics.

In [ ]:
import scipy # scipy is another package we need
my_linear_fit = scipy.stats.linregress(x, y)

In [ ]:
print(my_linear_fit)

The slope and intercept give you your line. The rvalue can be squared to give the r^2 value (goodness of fit). The p-value is another statistic that essentially tells you how likely it is that there is no correlation between these quantities (very small here!). Then theres errors on the slope (stderr) and intercept (intercept_stderr).

The way we can access this data is by asking for these properties of the linear fit *object* the function returns. This object has all of these things as properties that we can get by typing things like `my_linear_fit.intercept`, etc.

We can now use this to plot our fit against our data!

In [ ]:
def my_fit(x):
    return my_linear_fit.slope*x + my_linear_fit.intercept

plt.scatter(x,y, marker='x',color='k')
plt.plot(np.linspace(0,100,100),my_fit(np.linspace(0,100,100)), color='r', linewidth=3)

Now we can check our fit. If it is good then there should be roughly an even distribution of points above and below the line. Let's see if that's true by checking the *residuals* of the data (the difference between the actual and predicted values, treating x as the independent variable)

In [ ]:
dy = y - my_fit(x) #our residuals
#let's just pring out the number of positive vs negative residuals
print(len(dy[(dy>0)]), len(dy[(dy<0)]))

Not too bad! Very close! We can also take a closer look at the distribution of residuals, just for fun. We will use the hist() function in matplotlib which produces a histogram (binned numbers). It takes as an argument one set of data, as well as a number of bins to split it up into. You may also give it a `range=(min,max)`.

In [ ]:
 # you can play around with the number of bins. What happens if you make them too small?
plt.hist(dy, bins=20)

For your lab reports, it will always be important to report the goodness of any fit you perform. In the case of linear regression, this is the rvalue^2

In [ ]:
print('r-squared value:', my_linear_fit.rvalue**2)

That's very close to one! That's very good!

Let's see what happens when we fit random data (i.e. trying to find a correlation when there is none). In this case, we are going to assign 100 random numbers some other independent variable. This variable isn't random but just increasing, but this doesn't matter.

In [ ]:
rand_num = np.random.rand(100) #100 random numbers
ind_var = np.arange(100) #0,1,2...99
my_new_linear_fit = scipy.stats.linregress(ind_var, rand_num)

In [ ]:
my_new_linear_fit

Wow. So that slope is very close to zero, and that pvalue is now pretty darn high! This means that it is likely there is no correlation (which of course there isn't!)

In [ ]:
plt.scatter(ind_var, rand_num, marker='x', color='k')
def my_new_fit(x):
    return my_new_linear_fit.slope*x + my_new_linear_fit.intercept

plt.plot(np.linspace(0,100,100),my_new_fit(np.linspace(0,1,100)),color='r')

Now let's take a look at the residuals again

In [ ]:
#Let's look at the residuals again
dy_new = rand_num - my_new_fit(x) #our residuals
#let's just pring out the number of positive vs negative residuals
print(len(dy_new[(dy_new>0)]), len(dy_new[(dy_new<0)]))

In [ ]:
plt.hist(dy_new, bins=20)

In [ ]:
# And finally the r^2 value

print(my_new_linear_fit.rvalue**2)

Wow that's very small! This isn't a good fit... the data is random so that is what we'd expect!

## Error Bars

If your data has some error bars associated with it you can plot those too using `plt.errorbar(x, y, xerr=dx, yerr=dy)`.

We will typically assume that the errors/uncertainties  in data are *symmetric* and *gaussian*.

In [ ]:
xdata = [10,13.3, 16.2, 33.4]
ydata = [2, 13, 18.9, 20.2,]
xerror = [0.5, 0.3, 4, 5]
yerror = [2, 0.5, 6, 10]
#by default this will plot a line. If you want a scatter plot set linestyle=''
plt.errorbar(xdata, ydata, xerr=xerror, yerr=yerror, linestyle='')

# Mean, Standard Deviation, and Gaussian Distributions

The *Gaussian* distribution is crucial for error analysis. It is an exmple of a *probability distribution* representing the likelihood that a value devaites from some mean or predicted value. When you state that your error is +/- some number, the implication is that you are saying something about how wide (uncertain) or narrow (very certain) this distribution is. For small errors, you are saying that it is very *unlikely* that the true value deviates significantly from your measured result. When you see plots with error bars like the one above, it is very often the case that the errors are meant tO represent something like a Gaussian distribution of possible true answers relative to the measured result presented ("we measure this value but we cannot really rule out anything in this region")

The *Gaussian* or *Normal* distribution has the following mathematical form $f(x) = \frac{1}{\sigma \sqrt{2\pi}} e^{-\frac{1}{2}\left(\frac{x-\mu}{\sigma}\right)^2}$

Where $\mu$ is the mean or measured value and $\sigma$ is the *standard deviation*, represending how wide or narrow the distribution is. This $\sigma$ is what is typically showed as error bars in plots, or the uncertainty associated with a measurement. "This region is within one standard deviation away from the measured/mean value".

## But what is a probability distribution?

Given a probability distribution function $f(x)$, $f(x)dx$ represents the probability that x is between x and x+dx. More generally we can write probability that x is between value $a$ and $b$ as

$\int_a^b f(x)dx$

The probability of something being true has to be between 0 (never true) and 1 (always true). So every probability distribution is *normalized* such that the integral over the entire allowed range of $x$ is 1 (the true values must be *somewhere*). So, for a value $x$ that can be any real number:

$\int_{-\infty}^{\infty} f(x)dx = 1$

Below I define a function that will give the value of a Gaussian probability distribution for a given mean and standard deviation.

In [ ]:
def gaussian(x, sigma=10, mean=0):
    return 1/(sigma*np.sqrt(2*np.pi)) * np.exp(-0.5*((x-mean)/sigma)**2)

Numpy has a useful built-in function for sampling a normal (gaussian) distribution, called `np.random.normal()`.

Below we will sample this normal distribution for a fixed number of times, starting with a mean of 0 and a $\sigma=10$ (but you can set these to whatever you want!)

Note that below in the `plt.hist()` function we have the keyword `density=True`. This *normalizes* the distribution so that the integral is 1. In other words, it turns a binned number count (regular histogram) into a *probability distribution* so that we can compare it to a Normal (Guassian) distribution. It does this by dividing each individual bin number count by the total number of samples (`N_sample`) and the size of the given bin range.

In a lot of science, normalizing histograms like this is very useful because it makes the values more generally meaningful, rather than dependent on the number of samples you have. The important thing isn't the raw number of counts, but relative difference between counts at different values. By normalizing like this, one could compare different distributions that have different sample numbers.

In [ ]:
N_sample = 100
sigma=30
mean=30

rand_normal  = np.random.normal(loc=mean, size=N_sample, scale=sigma)

plt.hist(rand_normal, bins=20, density=True)
plt.plot(np.linspace(mean-(5*sigma),mean+(5*sigma),1000), 
         gaussian(np.linspace(mean-(5*sigma),mean+(5*sigma),1000), sigma=sigma, mean=mean))

Try the above for different means and sigmas and see what it looks like. It actually doesn't look great does it?

One thing to consider is that the standard deviation ($\sigma$) has an actual meaning when it comes to probabilities. One standard deviation on either side of the mean (i.e. the range $(\mu-\sigma, \mu+\sigma)$ has a probability of about 0.68 (or 68\%). 

Let's see what it is for our distribution. Note that I use the function `np.abs()` which takes the absolute value, rather than writing the range like I did above. It is just shorter!

In [ ]:
len(rand_normal[(np.abs(rand_normal-mean)<sigma)])/N_sample

Now, this is random, so I don't know exactly what you'll get. But if you try it a few times with `N_sample =100` you'll get some variation of this fraction. 

**Now, try to increase `N_sample` to values like 1000 and 10000. What happens to your plot and to your fraction of numbers within one standard deviation?**

If one sigma means 67% likelihood, what happens if we go out to $2\sigma$?

In [ ]:
len(rand_normal[(np.abs(rand_normal-mean)<2*sigma)])/N_sample

And now $3\sigma$?

In [ ]:
len(rand_normal[(np.abs(rand_normal-mean)<3*sigma)])/N_sample

You should see that $2\sigma$ encompasses roughly 95.5\% of the points and $3\sigma$ about 99.7\%. When you are thinking of your data and the meaning behind your error bars, keep this in mind. Usually errors are expressed in terms of $\sigma$. So "one-sigma", or the area encompassed by an error bar, represents a likelihood of about 67% that the "true" answer is within that range. and $3\sigma$ is 99\%. So, it isn't wild to think a number might lie outside of the one-sigma region... but three-sigma? Now there should only be about a 3 in 1000 chance that the truth lies outside of this region.

# Counting Errors

Counting Error/Shot Noise/Poisson Noise is how to (approximately) quantify the uncertainty in sampling the true likelihood of an event. Given a true underlying probabiliy of an event occuring, you might expect to see the event happen $M$ times out of $K$ tries. This quantifies the likelihood of instead detecting $N$ events.

This likelihood is governed by the Poison distribution. We won't worry about the functional form here, though for large samples it approaches a Gaussian distribution. The uncertainty (or "shot noise") in an observed count of $N$ events is $\sqrt{N}$. Should I observe $N$ events, I would have had a reasonable chance of having observed $N\pm\sqrt{N}$ events.

If we think about signal-to-noise - number of detections (signal) divided by this "shot noise" we get $\frac{N}{\sqrt{N}} = \sqrt{N}$. So as the number of detections increases we become more and more condident that we are sampling the true probability.

You will often see such errors when trying to measure a probability (fraction of events that meat some criteria given a sample of N total events). If the number of events that meat your criteria (say the number of heads from a coin flip) is $M$ and the number of total events observed is  $N$ than your measured probability is $M/N$ with an uncertainty of $\frac{\sqrt{M}}{N}$.

# Exercises

a) Use the `np.random.rand()` function to generate uniformly distributed random numbers between 0 and 1. Do this for a variety of different sample sizes spanning 10 to 1,000,000. For each sample, estimate the measured probability of finding a number less than 0.5. Plot this as a function of the number sampled and provide the appropriate errors. Do this as a scatter plot using `plt.errorbar()`. You will have to re-scale your axes for this! Use `plt.xscale('log')` to make your x-axis (number of samples) in log scale.

b) Provided below is a code that simulates dice rolls of variable sided dice. Generate large numbers of dice rolls with different combinations of multiple dice (e.g. 1 d6 + 1 d8, 3 d6 + 1 d10, etc), summing their results. Plot the *probability distribution* of the values you find. Calculate the mean and standard deviation of your sample and compare your measured probability distribution to a gaussian using the function provided above. Make plots showing this comparison, with the measured mean and standard deviations shown by verticle lines.

In [ ]:
def dice_roll(n,d=6):
    '''
    Code by Mark Kennedy
    Rolls a dice of face=d n times, and returns the results. 
    n : integer
    d : integer
    returns: array of results for n tosses.
    '''
    rng = np.random.default_rng()
    rolls = np.floor(d*rng.random((n,)))+1 # Have to add one, as the function returns rolls from interval [0,d-1]
    return rolls